In [1]:
from san_utils import *

import moabb,sys,pickle,os, openpyxl
from moabb.datasets import BNCI2014001
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank
from sklearn.model_selection import train_test_split


import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_samples,silhouette_score,calinski_harabasz_score,davies_bouldin_score


In [2]:
dataset = BNCI2014001()
dataset_name="BNCI2014001"
subjects = [1, 2, 3,4,5,6,7,8,9]; n_class=4 ; sfreq=250 
fmin = 4;fmax = 40; 
bandwidths=[[4],[2,4],[2,4,6],[2,4,6,8]]
BW=bandwidths[0]
allfilters=getfilters(fmin,fmax,bandwidth=BW) #from MS NN IV 
# a=list(allfilters)
# a.append([8.,30.])
# a.append([7.,35.])
# allfilters=np.array(a)
key_str = ''.join(map(str, BW))
print("key:",key_str, "|| BW:",BW )

# pkl_sdirec=dataset_name+"/tw_fb"+key_str+"/"
# pkl_sdirec

key: 4 || BW: [4]


In [7]:
##### Time window select ######
paradigm = MotorImagery(fmin=fmin,fmax=fmax,n_classes=n_class)
X, y, meta = paradigm.get_data(dataset=dataset, subjects=[1])
le,le_label_map=label_map_fn(y)

last_sec=int(X.shape[2]/sfreq)
timewindows=get_timewin(0,last_sec) #[(0,2),(0,2.5),(0,3),(0,3.5),(0,4),(0.5,2.5),(0.5,3),(0.5,3.5),(0.5,4),(1,3),(1,3.5),(1,4),(1.5,3.5),(1.5,4),(2,4)]
timewin={};all_timewin={};dscore_tw={}
for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])

    select_indices = list(np.where(meta["session"] == "session_T")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_E")[0])
    X1=X[select_indices,:];y1=y[select_indices]
    y_train = le.transform(y0);y_test = le.transform(y1)
    X_train=X0;X_test=X1
    # print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    
    timewin[subject],all_timewin[subject], dscore_tw[subject]=best_time_dscoredm(timewindows,X_train,y_train,sfreq)#dscore based selection

Choosing from all possible events


subject: 1
TW,Sscore: [0. 2.] 0.6177834863739855
TW,Sscore: [0.5 2.5] 0.7667255116029306
TW,Sscore: [1. 3.] 0.7508039013290515
TW,Sscore: [1.5 3.5] 0.6635396665475298
TW,Sscore: [2. 4.] 0.6046072037145992
TW,Sscore: [0.  2.5] 0.6942727631599379
TW,Sscore: [0.5 3. ] 0.8085084016116916
TW,Sscore: [1.  3.5] 0.7603723801773049
TW,Sscore: [1.5 4. ] 0.6777184999989615
TW,Sscore: [0. 3.] 0.7466225228467044
TW,Sscore: [0.5 3.5] 0.8179564982177778
TW,Sscore: [1. 4.] 0.7678709464590983
TW,Sscore: [0.  3.5] 0.766013189511637
TW,Sscore: [0.5 4. ] 0.8235874881290692
TW,Sscore: [0. 4.] 0.7798029246808948
best Time window: [0.5 4. ]
subject: 2
TW,Sscore: [0. 2.] 0.2194742416252107
TW,Sscore: [0.5 2.5] 0.25891743130725403
TW,Sscore: [1. 3.] 0.27900911888130875
TW,Sscore: [1.5 3.5] 0.28160621192861784
TW,Sscore: [2. 4.] 0.28326400725879947
TW,Sscore: [0.  2.5] 0.25055586519439466
TW,Sscore: [0.5 3. ] 0.2681179278848361
TW,Sscore: [1.  3.5] 0.3003128612439541
TW,Sscore: [1.5 4. ] 0.30797451181008584
TW,

In [8]:
###save 
with open(pkl_sdirec+'timewin.pkl', 'wb') as file:
    pickle.dump(timewin, file)   
with open(pkl_sdirec+'all_timewin.pkl', 'wb') as file:
    pickle.dump(all_timewin, file)
with open(pkl_sdirec+'dscore_tw.pkl', 'wb') as file:
    pickle.dump(dscore_tw, file)
pkl_sdirec

'BNCI2014001/tw_fb4/'

In [3]:
### load
with open(pkl_sdirec+'timewin.pkl', 'rb') as file:
    timewin=pickle.load(file)
with open(pkl_sdirec+'all_timewin.pkl', 'rb') as file:
    all_timewin=pickle.load( file)   
with open(pkl_sdirec+'dscore_tw.pkl', 'rb') as file:
    dscore_tw=pickle.load( file)

In [10]:
#### freq band select ######
def savepkl(pkl_sdirec,subject,sdict):
    sfilename=pkl_sdirec+str(subject)+'_scores.pkl'
    with open(sfilename, 'wb') as file:
        pickle.dump(sdict, file)
    print("saved pkl file!")
    return
    
nband_Sel=np.arange(1,len(allfilters)+1)
sub_filters={};all_filters={};dscore_fb={}
paradigm = FilterBankMotorImagery(filters=allfilters,n_classes=n_class)
for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])
    s1=int(sfreq*timewin[subject][0]);     s2=int(sfreq*timewin[subject][1]) ; X=X[:,:,s1:s2]
    
    select_indices = list(np.where(meta["session"] == "session_T")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_E")[0])
    X1=X[select_indices,:];y1=y[select_indices]
    
    y0 = le.transform(y0);y1 = le.transform(y1)

    filters={}
    for nb in nband_Sel:
        print("nband:",nb)
        filters[nb],all_filters[subject],dscore_fb[subject]=bestfilters_dscoredm(allfilters,X0,y0,nb)#dscore based selection
    sub_filters[subject]=filters
    sdict={"all_filters":all_filters,"dscore_fb":dscore_fb, "sub_filters":sub_filters,
           "le_label_map":le_label_map,"X0":X0, "y0":y0, "X1":X1, "y1":y1}
    savepkl(pkl_sdirec,subject,sdict)

Choosing from all possible events


subject: 1
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nband: 6
nband: 7
nband: 8
nband: 9
nband: 10
nband: 11
nband: 12
nband: 13
nband: 14
nband: 15
nband: 16
nband: 17
saved pkl file!
subject: 2
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nband: 6
nband: 7
nband: 8
nband: 9
nband: 10
nband: 11
nband: 12
nband: 13
nband: 14
nband: 15
nband: 16
nband: 17
saved pkl file!
subject: 3
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nband: 6
nband: 7
nband: 8
nband: 9
nband: 10
nband: 11
nband: 12
nband: 13
nband: 14
nband: 15
nband: 16
nband: 17
saved pkl file!
subject: 4
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nband: 6
nband: 7
nband: 8
nband: 9
nband: 10
nband: 11
nband: 12
nband: 13
nband: 14
nband: 15
nband: 16
nband: 17
saved pkl file!
subject: 5
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nband: 6
nband: 7
nband: 8
nband: 9
nband: 10
nband: 11
nband: 12
nband: 13
nband: 14
nband: 15
nband: 16
nband: 17
saved pkl file!
subject: 6
nband: 1
nband: 2
nband: 3
nband: 4
nband: 5
nban

In [11]:

########### save filters, dscore in .pkl file and .csv file #######
with open(pkl_sdirec+'le_label_map.pkl', 'wb') as file:
    pickle.dump(le_label_map, file)

with open(pkl_sdirec+'all_filters.pkl', 'wb') as file:
    pickle.dump(all_filters, file)
with open(pkl_sdirec+'dscore_fb.pkl', 'wb') as file:
    pickle.dump(dscore_fb, file)
with open(pkl_sdirec+'sub_filters.pkl', 'wb') as file:
    pickle.dump(sub_filters, file)      
    

In [12]:
############### load .pkl file #############3
with open(pkl_sdirec+'le_label_map.pkl', 'rb') as file:
    le_label_map=pickle.load( file)
    
with open(pkl_sdirec+'all_filters.pkl', 'rb') as file:
    all_filters=pickle.load(file)
with open(pkl_sdirec+'dscore_fb.pkl', 'rb') as file:
    dscore_fb=pickle.load( file)   
with open(pkl_sdirec+'sub_filters.pkl', 'rb') as file:
    sub_filters=pickle.load( file)


In [13]:
path = pkl_sdirec+'alltwfilters_dscore'+key_str+'.xlsx'

#############1 save excel all ######## https://xlsxwriter.readthedocs.io/example_pandas_multiple.html
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_le=pd.DataFrame([le_label_map])
df_le.to_excel(writer, sheet_name="le_map",index=False)

df_fb=pd.DataFrame.from_dict(dscore_fb)
df_fb.insert(loc = 0, column = 'sub_band', value = all_filters[1])
df_fb.to_excel(writer, sheet_name="subband_dscoredm",index=False)

df_tw=pd.DataFrame.from_dict(dscore_tw)
df_tw.insert(loc = 0, column = 'time_win', value = all_timewin[1])
df_tw.to_excel(writer, sheet_name="timewin_dscoredm",index=False)

writer.save()
writer.close()





C:\Users\admin\anaconda3\envs\moabb\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [14]:
#############2 save excel best ######## https://xlsxwriter.readthedocs.io/example_pandas_multiple.html

path_best=pkl_sdirec+'Best_twfilters'+key_str+'.xlsx'
writer = pd.ExcelWriter(path_best, engine='xlsxwriter')

df_tw=pd.DataFrame()
df_tw["sub"]=timewin.keys()
df_tw["besttw"]=timewin.values()
df_tw.to_excel(writer, sheet_name="bestTW",index=False)

df_bfb = pd.DataFrame()
for sub in subjects:
    df_bfb["num_band"]=sub_filters[sub].keys()
    df_bfb[sub]=sub_filters[sub].values()
df_bfb.to_excel(writer, sheet_name="bestfb",index=False)

writer.save()
writer.close()



C:\Users\admin\anaconda3\envs\moabb\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
